# Find semantic units of interest (IoU) by running the implementation of the Semantic pipeline

### Import

In [1]:
import numpy as np
import tensorflow as tf

from utils.load_config import load_config
from utils.extraction_model import load_extraction_model

from utils.Semantic.load_coco_semantic_annotations import load_coco_semantic_annotations
from utils.Semantic.load_coco_semantic_annotations import get_coco_cat_ids
from utils.Semantic.find_semantic_units import find_semantic_units
from utils.Semantic.find_semantic_units import get_IoU_per_category

In [2]:
# define configuration
config_path = 'SEM_t01_find_semantic_units_m0001.json'

# load config
config = load_config(config_path, path='../../configs/Semantic')

## Load Model

In [3]:
# load CNN model to compute IoU on it
model = load_extraction_model(config, input_shape=tuple(config["input_shape"]))
model = tf.keras.Model(inputs=model.input, outputs=model.get_layer(config['v4_layer']).output)
size_ft = tuple(np.shape(model.output)[1:3])
print("[LOAD] size_ft", size_ft)
print("[LOAD] Model loaded")

1 Physical GPUs, 1 Logical GPUs
Metal device set to: Apple M1 Pro


2022-07-13 11:56:38.334287: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-13 11:56:38.334427: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


[LOAD] size_ft (56, 56)
[LOAD] Model loaded


## Load Data

In [4]:
semantic_data = load_coco_semantic_annotations(config)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


## Find semantic units

In [5]:
# get all idx list for each category
sem_idx_list = find_semantic_units(model, semantic_data, config, verbose=1)

# get category IDs of interest (transform semantic units name to category id of COCO)
cat_ids = get_coco_cat_ids(config, config['semantic_units'], to_numpy=True)

# get CNN feature map indexes (gather all feature map index across the whole architecture for each category of interest)
cat_feature_map_indexes = get_IoU_per_category(sem_idx_list, cat_ids)

print("-- finished computing semantic units --")

  0%|          | 0/9 [00:00<?, ?it/s]

2/2 [==============================] - 0s 63ms/step


2022-07-13 11:56:41.258081: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-13 11:56:41.290054: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
 11%|█         | 1/9 [00:53<07:11, 53.90s/it]

2/2 [==============================] - 0s 106ms/step


2022-07-13 11:57:35.146684: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
 22%|██▏       | 2/9 [01:47<06:16, 53.83s/it]

...

1/2 [==============>...............] - ETA: 0s

2022-07-13 12:14:55.259142: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 563ms/step


100%|██████████| 9/9 [22:42<00:00, 151.41s/it]

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
-- finished computing semantic units --


## Print Results

In [6]:
categories = ["eyes", "eyebrow", "nose", "lips"]  # this is taken from Hasty.ai labelling
for i, cat in enumerate(cat_feature_map_indexes):
    print("{} ({}) indices found: ".format(cat, categories[i]))
    print(cat_feature_map_indexes[cat][config['v4_layer']]['indexes'])

category_10 (eyes) indices found: 
[161 114 229  82  19 125 136]
category_13 (eyebrow) indices found: 
[ 68 125]
category_17 (nose) indices found: 
[101  52 172 210 167 216  85 102 232   4  77 162   1  33 208  29  30  93
  59 104  23 133 161 115  19  81  74  97 148 120 221   9 103 149 246 150
 137 226  34 233 229  54  31 242  37 155 171 255 235 124 224 196 227  27
 241 107 228 211 243 166  20]
category_20 (lips) indices found: 
[235 203  68 125   3 181 197   2  87 240   6  95  60 157 227 111]
